In [1]:
import pandas as pd
import numpy as np

from os.path import exists, join
from os import makedirs

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score


Подсказка: `pd.join()`

In [2]:
train_data = pd.read_csv('data/train/train0.csv')
x_train = train_data[train_data.columns[2:]]
y_train = train_data['Category']

In [6]:
len(x_train)

284

Здесь используем кросс-валидацию. [Подробнее](http://scikit-learn.org/stable/modules/cross_validation.html)

In [28]:
clf = RandomForestClassifier(n_estimators=5, max_depth=5, criterion='entropy')
scores = cross_val_score(clf, x_train, y_train, cv=10, scoring='f1_macro')
print(scores)
print("F1 score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.92676768  1.          0.92368421  0.92368421  0.92676768  0.85353535
  0.84444444  0.92222222  0.87819549  0.84117647]
F1 score: 0.90 (+/- 0.09)


In [56]:
import xgboost as xgb
clf_xgboost = xgb.XGBClassifier(learning_rate=0.08, n_estimators=450,gamma=3)
clf_xgboost.fit(x_train, y_train)
scores1 = cross_val_score(clf_xgboost, x_train, y_train, cv=10, scoring='f1_macro')
print(scores)
print(scores.mean(), scores.std() * 2)

[ 0.92676768  1.          0.92368421  0.92368421  0.92676768  0.85353535
  0.84444444  0.92222222  0.87819549  0.84117647]
0.90404777541 0.093948731651


In [38]:
clf_xgboost = xgb.XGBClassifier(max_depth=2, learning_rate=0.05, n_estimators=250)
clf_xgboost.fit(x_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=250, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [29]:
clf = RandomForestClassifier(n_estimators=10)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [57]:
test_data = pd.read_csv('data/test/test0.csv')
x_test = test_data[test_data.columns[1:]]

In [58]:
y_test = clf_xgboost.predict(x_test)

In [13]:
y_test

array([1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1])

In [59]:
results_dir = 'results/'
if not exists(results_dir):
    makedirs(results_dir)

pd.DataFrame({
    'ID': test_data['ID'],
    'Category': y_test
}).to_csv(join(results_dir, 'submission.csv'), index=False, columns=['ID', 'Category'])

Данные взяты из открытого источника.  
**Авторы**: Lichman, M. (2013). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.